# Generation of wrongdoings emails

In [ ]:
#Install transformers library
#!pip install transformers

In [ ]:
#Import login library to load models
from huggingface_hub import login

In [ ]:
#Put your credentials
login("...")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#Import torch, model class, tokenizer class
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#Load Gemma tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
#Load Gemma model
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", pad_token_id=tokenizer.eos_token_id).to(torch_device)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
#Request to generate letters to the CEO
#with proposals for stretching valuation of a subsidiary
input_text = '''1. Generate a name for CEO.
2. Generate a name for a subsidiary company.
3. Generate a synonym for the expression "stretching valuation".
4. Generae a name for sender.
5. Write an email to convince CEO to "stretch valuation" of the subsidiary company
trying to demonstrate several quarters of increasing operating cashflow and reserves growth.'''

In [ ]:
#Provide prompt to model
model_inputs = tokenizer(input_text, return_tensors='pt').to(torch_device)

In [ ]:
#Set seed to reproduce results. Change the seed though to get different results
from transformers import set_seed

set_seed(150)

#Set top_k and set top_p =  and num_return_sequences
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=1000,
    do_sample=True,
    temperature=2.0,
    top_k=100,
    top_p=0.95,
    num_return_sequences=20
)

In [ ]:
#Check generated data
print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 1. Generate a name for CEO.
2. Generate a name for a subsidiary company.
3. Generate a synonym for the expression "stretching valuation".
4. Generae a name for sender.
5. Write an email to convince CEO to "stretch valuation" of the subsidiary company
trying to demonstrate several quarters of increasing operating cashflow and reserves growth.
6. Write a poem about loneliness in the work place.

**Additional Requirements**
- Ensure CEO initials always come before their last name (e.g. J. Obama = Jeanne O'Malley).
- Avoid common terms (“chairman”,” ”CEO” etc.)and slang.
- Use creative, semantic, descriptive words that show CEO intellect and character.

**Job Description**: Develop professional sounding communication strategies aligned with establishing best practices, driving long-term profitability, motivating company performance and generating shareholder value.
1: 1. Generate

In [ ]:
#Import pandas
import pandas as pd

In [ ]:
#Load generated data to a dataframe, split by word "Dear"
data = []
for i, sample_output in enumerate(sample_outputs):
  sample = ("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
  extract = sample.split("Dear")
  try:
    data.append(("Dear" + extract[1]))
  except:
    pass
df = pd.DataFrame(data)

In [ ]:
#Load dataframe to csv file
df.to_csv("data_wrongdoings.csv", encoding='utf-8', index=False)